In [ ]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

import os
import re
import json
import glob
from collections import defaultdict
from textblob import TextBlob
from functools import partial

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

import nltk
import spacy
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])
nlp.max_length = 4000000
from nltk.probability import FreqDist
from wordcloud import WordCloud, STOPWORDS

from tqdm.autonotebook import tqdm
import string

%matplotlib inline

os.listdir('/kaggle/input/coleridgeinitiative-show-us-the-data/')

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold

In [ ]:
# reading csv files and train & test file paths
train_df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')
sample_sub = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
train_files_path = '../input/coleridgeinitiative-show-us-the-data/train'
test_files_path = '../input/coleridgeinitiative-show-us-the-data/test'

In [ ]:
dirname = "/kaggle/input/coleridgeinitiative-show-us-the-data"
#df = pd.read_csv(os.path.join(dirname, "train.csv"))

In [ ]:
# Credit to: https://www.kaggle.com/c/coleridgeinitiative-show-us-the-data/discussion/230341
df = pd.read_pickle("../input/ner-coleridge-initiative/show_us_the_data_train_ner.pkl")

In [ ]:
df

In [ ]:
df["token"].nunique()

In [ ]:
df["sentence#"].value_counts(dropna=False)

In [ ]:
df["sentence#"].unique()

In [ ]:
df["sentence"].value_counts(dropna=False)

In [ ]:
df["sentence"].unique()

In [ ]:
pd.options.display.max_rows = 60 #9999

In [ ]:
df[df["sentence"]=="sentence9999"]

In [ ]:
df[df["sentence"]=="sentence0"]

#### Data Preparation for NERD fine-tuning
- data structure:  
    {'sentences': [[], [], ..., []],  
     'tags': [[], [], ..., []]}

In [ ]:
#tmp = df.groupby("sentence")["entity"].count()

In [ ]:
df

In [ ]:
# CV (GroupKfold)
"""example
import numpy as np
from sklearn.model_selection import GroupKFold
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
y = np.array([1, 2, 3, 4])
groups = np.array([0, 0, 2, 2])
group_kfold = GroupKFold(n_splits=2)
group_kfold.get_n_splits(X, y, groups)

print(group_kfold)

for train_index, test_index in group_kfold.split(X, y, groups):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(X_train, X_test, y_train, y_test)
"""

X = df.index.values
y = df["entity"].values
groups = df["sentence"].values

group_kfold = GroupKFold(n_splits=5)
group_kfold.get_n_splits(X, y, groups)

print(group_kfold)

res = {}
for i, (train_index, test_index) in enumerate(group_kfold.split(X, y, groups)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    res[i] = [X_train, y_train, X_test, y_test]

In [ ]:
df

In [ ]:
# train / dev split
#train, dev = train_test_split(df, test_size=0.3, random_state=42, shuffle=True, stratify=df["sentence"])

In [ ]:
train

In [ ]:
dev